# Monotone schemes with a second order non-linearity, in two space dimensions

This notebook illustrates the use of monotone finite difference schemes to compute viscosity solutions of non-linear PDEs, in two space dimensions. 
We consider the operator
$$
    \Lambda u(x) := \alpha(x) \lambda_{\max}(\nabla^2 u(x)) + \lambda_{\min}(\nabla^2 u(x))
$$
in the PDE
$$
    \Lambda u(x) = \beta(x),
$$
with Dirichlet boundary conditions. The PDE parameters are a positive function $\alpha$, and an arbitrary function $\beta$.
We denote by $\lambda_{\max}(M)$ and $\lambda_{\min}(M)$ are the largest and smallest eigenvalue of a positive definite tensor $M$. More details on this problem below.

We design a two monotone numerical schemes: 
* The first sheme, based on a discretization of the space of controls, is simple to implement. However it is quite costly numerically, and it induces a consistency defect.
* The second scheme is second order consistent and possibly cheaper numerically. However, implementation details are more subtle.

The two schemes involves adaptive stencils, built using techniques from lattice geometry. The techniques developed are fairly general, and can be applied to a wide rande of non-linear PDEs. Numerical implementation is kept simple thanks to the use of automatic differentiation.

### Reformulation and an extremal operator

Assume without loss of generality that $\alpha \leq 1$. Then for any positive definite matrix $M$ one has 
$$
    \alpha \lambda_{\max}(M) + \lambda_{\min}(M) = \min_{0 \leq \theta \leq \pi} {\rm tr}(D_\alpha(\theta) M),
$$
where we denoted, with $e(\theta) := (\cos \theta, \sin \theta)$
$$
    D_\alpha(\theta) = \alpha\, e(\theta) e(\theta)^T + e(\theta)^\perp (e(\theta)^\perp)^T,
$$
the symmetric matrix whose eigenvalues are $\alpha$ and $1$, the former associated with the eigenvector $e(\theta)$.

**Remark on the range of the variable $\theta$.**
For any $\theta\in \mathbb R$, one has $e(\theta+\pi) = -e(\theta)$, and therefore $D_\alpha(\theta+\pi) = D_\alpha(\theta)$. By periodicity, one may therefore limit its attention to the interval $[0,\pi]$.

**Remark on the case $\alpha\geq 1$.**
This second case is handled by replacing the minimum over $\theta\in [0,\pi]$ with a maximum. This does not induce any additional difficulty from the theoretical or numerical standpoints. However, for the sake of simplicity, we make the assumption that $\alpha\leq 1$ in the following.

### First discretization strategy (sampling of the control space)

Let $K$ be a positive integer, and let $\theta_1 \leq \cdots \leq \theta_K$ be a sampling of the interval $[0,\pi]$. Then we may consider the approximate operator
$$
    \Lambda_K u(x) := \min_{1 \leq k \leq K} {\rm tr} (D_\alpha(\theta_k) \nabla^2 u(x)).
$$
Introduce decompositions of the tensors, obtained e.g. by Selling's method,
$$
    D_\alpha(\theta_k) = \sum_{1 \leq i \leq n} \mu_{ki} e_{ki} e_{ki}^T,
$$
where $\mu_{ki} \geq 0$ and $e_{ki}$ has integer coordinates. Then we obtain the monotone numerical scheme
$$
    \min_{1 \leq k \leq K} \sum_{1 \leq i \leq n} \mu_{ki} \frac{ u(x+h e_{ki}) - 2 u(x) +u(x-h e_{ki})} {h^2}.
$$
A consistency defect remains, which can be estimated in terms of the width of the sampling $\theta_1,\cdots,\theta_K$ of the control space $[0,\pi]$. 

An additional problem is that the numerical scheme cost increases as $K$ increases.
This issue becomes more acute in the case of a multi-dimensional control space.

### Second discretization (consistent implementation)

In order to introduce this discretization, we need to recall some elements from lattice geometry.
Selling's decomposition of a tensor $D$ involves a geometrical object, referred to as a *$D$-obtuse superbase* and here  denoted
$$
    {\rm osb}(D).
$$
The obtuse superbase $s={\rm osb}(D)$ dictates the support $(e_{si})_{i=1}^n$ of Selling's decomposition of $D$, hence the stencil of the numerical scheme. We can take advantage of this fact to rewrite the operator as 
$$
    \Lambda u(x) = \min_{s \in S} \Lambda_s u(x)
$$
where 
$$
    \Lambda_s u(x) := \min_{\theta, {\rm obs}(D_\alpha(\theta)) = s} {\rm tr} (D_\alpha(\theta) \nabla^2 u).
$$
Each operator $\Lambda_s$ admits the consistent discretization
$$
    \Lambda_s u(x) \approx \min_{\theta, {\rm obs}(D_\alpha(\theta)) = s} \sum_{1 \leq i \leq n} 
    \mu_{si}(\theta) \frac{u(x+h e_{si}) - 2 u(x) + u(x-e_{si})} {h^2}, 
$$
and a closed form can be obtained for the r.h.s. by examining a simple optimization problem.

## 0. Importing the required libraries

In [2]:
import sys; sys.path.append("..") # Allow import from parent directory
from NumericalSchemes import Selling
from NumericalSchemes import LinearParallel as lp
from NumericalSchemes import FiniteDifferences as fd
spad = fd.spAD # Alternatively : from NumericalSchemes import SparseAutomaticDifferentiation as spad

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg; import scipy.sparse; import scipy.sparse.linalg 

Some utility functions

In [4]:
def as_field(u,shape):
    ndim = len(shape)
    if u.ndim>=ndim and u.shape[-ndim:]==shape: return u
    else: return spad.broadcast_to(u.reshape(u.shape+(1,)*ndim), u.shape+shape)

In [5]:
def solve_residue(res):
    return - scipy.sparse.linalg.spsolve(
        scipy.sparse.coo_matrix(res.triplets()).tocsr(),
        res.value.flatten()).reshape(res.shape)

def solve(Scheme,params,guess,print_period=1,niter=8,relax=0.):
    u = spad.identity(guess.shape)+guess
    for i in range(niter):
        residue = Scheme(u,*params)
        if (i-1)%print_period ==0  or i==niter-1:
            print("Iteration :",i,", Residue norm :", LInfNorm(residue.value)) # Before solve
        u.value += solve_residue(residue + relax)
    return u.value

In [6]:
import importlib
fd = importlib.reload(fd)
spad = importlib.reload(spad)
lp = importlib.reload(lp)
Selling = importlib.reload(Selling)

## 1. Sampling based discretization